# Domain Adaptation Via Activation Shaping

**Teacher assistant** Iurada Leonardo

**Students**

- Bar Giorgio
- Distefano Giuseppe
- Incaviglia Salvatore

## 0 - Reproduce the Baseline

### Libraries

In [1]:
!pip install torch torchvision tqdm torchmetrics

import torch

import torch.backends.mps
from argparse import ArgumentParser

import os
import torchvision.transforms as T

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR10

import numpy as np
import random
from PIL import Image

import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

import torch.nn.functional as F
from torchmetrics import Accuracy
from tqdm import tqdm

import logging
import warnings

import argparse

### Setup PACS dataset and environment

In [2]:
# Download PACS Dataset Images and Labels
!git clone https://github.com/MachineLearning2020/Homework3-PACS/
!git clone https://github.com/silvia1993/DANN_Template/

# Setup data
!rm -rf data || true
!rm -rf record || true
!mkdir data
!mkdir data/kfold
!cp -r Homework3-PACS/PACS/ data/kfold
!cp DANN_Template/txt_lists/*.txt data
!rm -rf Homework3-PACS/
!rm -rf DANN_Template/

Cloning into 'Homework3-PACS'...
remote: Enumerating objects: 10032, done.
remote: Total 10032 (delta 0), reused 0 (delta 0), pack-reused 10032
Receiving objects: 100% (10032/10032), 174.13 MiB | 39.00 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (9993/9993), done.
Cloning into 'DANN_Template'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23
Receiving objects: 100% (23/23), 33.86 KiB | 1.30 MiB/s, done.
Resolving deltas: 100% (5/5), done.


### Globals

#### globals

In [3]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

CONFIG = dotdict({})

if torch.cuda.is_available():
    CONFIG.device = 'cuda'
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    CONFIG.device = 'mps'
else:
    CONFIG.device = 'cpu'

CONFIG.dtype = torch.float32

#### parse args

In [4]:
def _clear_args(parsed_args):
    parsed_args.experiment_args = eval(parsed_args.experiment_args)
    parsed_args.dataset_args = eval(parsed_args.dataset_args)
    return parsed_args


def parse_arguments():
    parser = ArgumentParser()

    parser.add_argument('--seed', type=int, default=0, help='Seed used for deterministic behavior')
    parser.add_argument('--test_only', action='store_true', help='Whether to skip training')
    parser.add_argument('--cpu', action='store_true', help='Whether to force the usage of CPU')

    parser.add_argument('--experiment', type=str, default='baseline')
    parser.add_argument('--experiment_name', type=str, default='baseline')
    parser.add_argument('--experiment_args', type=str, default='{}')
    parser.add_argument('--dataset_args', type=str, default='{}')

    parser.add_argument('--batch_size', type=int, default=128)
    parser.add_argument('--epochs', type=int, default=30)
    parser.add_argument('--num_workers', type=int, default=5)
    parser.add_argument('--grad_accum_steps', type=int, default=1)

    return _clear_args(parser.parse_args())

#### Global variables

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = 7
BATCH_SIZE = 256
LR = 1e-3               # The initial Learning Rate
MOMENTUM = 0.9          # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5     # Regularization, you can keep this at the default
NUM_EPOCHS = 30         # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20          # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1             # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

### Dataset

#### utils

In [6]:
class BaseDataset(Dataset):
    def __init__(self, examples, transform):
        self.examples = examples
        self.T = transform


    def __len__(self):
        return len(self.examples)


    def __getitem__(self, index):
        x, y = self.examples[index]
        x = Image.open(x).convert('RGB')
        x = self.T(x).to(CONFIG.dtype)
        y = torch.tensor(y).long()
        return x, y

######################################################
# TODO: modify 'BaseDataset' for the Domain Adaptation setting.
# Hint: randomly sample 'target_examples' to obtain targ_x
#class DomainAdaptationDataset(Dataset):
#    def __init__(self, source_examples, target_examples, transform):
#        self.source_examples = source_examples
#        self.target_examples = target_examples
#        self.T = transform
#
#    def __len__(self):
#        return len(self.source_examples)
#
#    def __getitem__(self, index):
#        src_x, src_y = ...
#        targ_x = ...
#
#        src_x = self.T(src_x)
#        targ_x = self.T(targ_x)
#        return src_x, src_y, targ_x

# [OPTIONAL] TODO: modify 'BaseDataset' for the Domain Generalization setting.
# Hint: combine the examples from the 3 source domains into a single 'examples' list
#class DomainGeneralizationDataset(Dataset):
#    def __init__(self, examples, transform):
#        self.examples = examples
#        self.T = transform
#
#    def __len__(self):
#        return len(self.examples)
#
#    def __getitem__(self, index):
#        x1, x2, x3 = self.examples[index]
#        x1, x2, x3 = self.T(x1), self.T(x2), self.T(x3)
#        targ_x = self.T(targ_x)
#        return x1, x2, x3

######################################################

class SeededDataLoader(DataLoader):
    def __init__(self, dataset: Dataset, batch_size=1, shuffle=None,
                 sampler=None,
                 batch_sampler=None,
                 num_workers=0, collate_fn=None,
                 pin_memory=False, drop_last=False, timeout=0,
                 worker_init_fn=None, multiprocessing_context=None,
                 generator=None, *, prefetch_factor=None, persistent_workers=False,
                 pin_memory_device=""):

        if not CONFIG.use_nondeterministic:
            def seed_worker(worker_id):
                worker_seed = torch.initial_seed() % 2**32
                np.random.seed(worker_seed)
                random.seed(worker_seed)

            generator = torch.Generator()
            generator.manual_seed(CONFIG.seed)

            worker_init_fn = seed_worker

        super().__init__(dataset, batch_size, shuffle, sampler, batch_sampler, num_workers, collate_fn,
                         pin_memory, drop_last, timeout, worker_init_fn, multiprocessing_context, generator,
                         prefetch_factor=prefetch_factor, persistent_workers=persistent_workers,
                         pin_memory_device=pin_memory_device)



#### PACS

In [7]:
def get_transform(size, mean, std, preprocess):
    transform = []
    if preprocess:
        transform.append(T.Resize(256))
        transform.append(T.RandomResizedCrop(size=size, scale=(0.7, 1.0)))
        transform.append(T.RandomHorizontalFlip())
    else:
        transform.append(T.Resize(size))
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean, std))
    return T.Compose(transform)


def load_data():
    CONFIG.num_classes = 7
    CONFIG.data_input_size = (3, 224, 224)

    # Create transforms
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) # ImageNet Pretrain statistics
    train_transform = get_transform(size=224, mean=mean, std=std, preprocess=True)
    test_transform = get_transform(size=224, mean=mean, std=std, preprocess=False)

    # Load examples & create Dataset
    #if CONFIG.experiment in ['baseline']:
    if CONFIG.experiment in ['baseline', 'activation_shaping_last', 'activation_shaping_one', 'activation_shaping_three', 'random']:
        source_examples, target_examples = [], []

        # Load source
        f = open(os.path.join(CONFIG.dataset_args['text_root'], f"{CONFIG.dataset_args['source_domain']}.txt"), 'r')
        for line in f:
            line = line.strip().split()
            path, label = line[0].split('/')[0:], int(line[1])
            source_examples.append((os.path.join(CONFIG.dataset_args['images_root'], *path), label))
        f.close()

        # Load target
        f = open(os.path.join(CONFIG.dataset_args['text_root'], f"{CONFIG.dataset_args['target_domain']}.txt"), 'r')
        for line in f:
            line = line.strip().split()
            path, label = line[0].split('/')[0:], int(line[1])
            target_examples.append((os.path.join(CONFIG.dataset_args['images_root'], *path), label))
        f.close()

        train_dataset = BaseDataset(source_examples, transform=train_transform)
        test_dataset = BaseDataset(target_examples, transform=test_transform)

    ######################################################
    #elif... TODO: Add here how to create the Dataset object for the other experiments


    ######################################################

    # Dataloaders
    train_loader = SeededDataLoader(
        train_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=True,
        num_workers=CONFIG.num_workers,
        pin_memory=True,
        persistent_workers=True
    )

    test_loader = SeededDataLoader(
        test_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=False,
        num_workers=CONFIG.num_workers,
        pin_memory=True,
        persistent_workers=True
    )

    return {'train': train_loader, 'test': test_loader}

### Models

#### ResNet

In [8]:
class BaseResNet18(nn.Module):
    def __init__(self):
        super(BaseResNet18, self).__init__()
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)


    def forward(self, x):
        return self.resnet(x)

######################################################

def get_activation_shaping_hook (mask):

    # Activation Shaping Module as a function that shall be hooked via 'register_forward_hook'
    # The hook captures mask variable from the parent scope, to update it,
    # remove() the hook and register a new one with the updated mask.

    def activation_shaping_hook(module, input, output):

        # print('ASH module registered on: ', module, 'mask shape: ', mask.shape, 'mask sum(): ', mask.sum())

        # binarize both activation map and mask using zero as threshold
        A_binary = torch.where(output<=0, torch.tensor(0.0), torch.tensor(1.0))
        M_binary = torch.where(mask<=0, torch.tensor(0.0), torch.tensor(1.0))

        # return the element-wise product of activation map and mask
        shaped_output = A_binary * M_binary
        return shaped_output

    return activation_shaping_hook

######################################################

# 'BaseResNet18' including the Activation Shaping Module
class ASHResNet18(nn.Module):
    def __init__(self):
        super(ASHResNet18, self).__init__()
        # self.resnet = resnet18(pretrained=False)
        self.resnet = resnet18(weights=ResNet18_Weights)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 7)

        self.hook_handles = {}

    def register_activation_shaping_hook(self, layer_name, mask):

        hook = get_activation_shaping_hook(mask)

        for name, module in self.resnet.named_modules():
            if ((isinstance(module, nn.ReLU) or isinstance(module, nn.Conv2d) or isinstance(module, nn.BatchNorm2d)) and name == layer_name):
                if not name in self.hook_handles:
                    print('Insert activation shaping layer after ', name, module)
                    self.hook_handles[name] = module.register_forward_hook(hook)

    def remove_activation_shaping_hook(self, name):
        if name in self.hook_handles and self.hook_handles[name] is not None:
            handle = self.hook_handles[name]
            handle.remove()

    def forward(self, x):
        return self.resnet(x)

    '''
    def activation_shaping_hook (self, module, input, output):
        shaped_output = output * self.mask
        return shaped_output

    def define_network (self, net_name='ash_last', mask=None, mask_out_ratio=0.1):
        if mask is None:
            # Create a mask tensor with a given ratio of zeros
            rand_mat = torch.rand(512, 7, 7).to(CONFIG.device)
            self.mask_out_ratio = mask_out_ratio
            mask = torch.where(rand_mat <= self.mask_out_ratio, 0.0, 1.0).to(CONFIG.device)
        self.mask = mask

        if net_name=='ash_last': # ASH module after last convolutional layer
            hook = get_activation_shaping_hook(mask)
            #self.hook_handle = self.resnet.layer4[1].relu.register_forward_hook(hook)
            self.hook_handle = self.resnet.layer4[0].conv2.register_forward_hook(self.activation_shaping_hook)

        elif net_name=='ash_one': # ASH module after each convolutional layer
            self.hook_handles = []
            layers = [self.resnet.layer1, self.resnet.layer2, self.resnet.layer3, self.resnet.layer4]
            for layer in layers:
                for module in layer.children():
                    if isinstance(module, nn.Conv2d):
                        hook = get_activation_shaping_hook(mask)
                        hook_handle = module.register_forward_hook(hook)
                        self.hook_handles.append(hook_handle)

        elif net_name=='ash_three': # ASH module after every three convolutional layers
            self.hook_handles = []
            layers = [self.resnet.layer1, self.resnet.layer2, self.resnet.layer3, self.resnet.layer4]
            for i, layer in enumerate(layers):
                for module in layer.children():
                    if isinstance(module, nn.Conv2d):
                        if (i%3 == 0):
                            hook = get_activation_shaping_hook(mask)
                            hook_handle = module.register_forward_hook(hook)
                            self.hook_handles.append(hook_handle)


    def register_activation_shaping_hook(self, layer_name = 'layer4.1.relu', mask_out_ratio = 0.0):
        self.layer_name = layer_name
        self.mask_out_ratio = mask_out_ratio

        # create a mask tensor with a given ratio of zeros
        print('self.mask_out_ratio: ', self.mask_out_ratio)
        rand_mat = torch.rand(512, 7, 7).to(CONFIG.device)
        mask = torch.where(rand_mat <= self.mask_out_ratio, 0.0, 1.0).to(CONFIG.device)

        hook = get_activation_shaping_hook(mask)
        # penultimate_layer = self.resnet.layer4[1].bn2
        # self.hook_handle = penultimate_layer.register_forward_hook(hook)

        for name, module in self.resnet.named_modules():
          if (isinstance(module, nn.ReLU) and name == self.layer_name):
            print('Insert activation shaping layer after ', name, module)
            self.hook_handle = module.register_forward_hook(hook)

    def remove_activation_shaping_hook (self):
        if self.hook_handle is not None:
            self.hook_handle.remove()

    def remove_activation_shaping_hooks (self):
        if self.hook_handles is not None:
            for hook_handle in self.hook_handles:
                hook_handle.remove()

    def forward(self, x):
        return self.resnet(x)

    '''

######################################################

### Run

Evaluate

In [9]:
@torch.no_grad()
def evaluate(model, data):
    model.eval()

    acc_meter = Accuracy(task='multiclass', num_classes=CONFIG.num_classes)
    acc_meter = acc_meter.to(CONFIG.device)

    loss = [0.0, 0]
    for x, y in tqdm(data):
        with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):
            x, y = x.to(CONFIG.device), y.to(CONFIG.device)
            logits = model(x)
            acc_meter.update(logits, y)
            loss[0] += F.cross_entropy(logits, y).item()
            loss[1] += x.size(0)

    accuracy = acc_meter.compute()
    loss = loss[0] / loss[1]
    logging.info(f'Accuracy: {100 * accuracy:.2f} - Loss: {loss}')
    print(f'Accuracy: {100 * accuracy:.2f} - Loss: {loss} \n')

Train

In [10]:
def train(model, data):
    # Create optimizers & schedulers
    optimizer = torch.optim.SGD(model.parameters(), weight_decay=0.0005, momentum=0.9, nesterov=True, lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=int(CONFIG.epochs * 0.8), gamma=0.1)
    scaler = torch.cuda.amp.GradScaler(enabled=True)

    # Load checkpoint (if it exists)
    cur_epoch = 0
    if os.path.exists(os.path.join('record', CONFIG.experiment_name, 'last.pth')):
        checkpoint = torch.load(os.path.join('record', CONFIG.experiment_name, 'last.pth'))
        cur_epoch = checkpoint['epoch']
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        model.load_state_dict(checkpoint['model'])

    # Optimization loop
    for epoch in range(cur_epoch, CONFIG.epochs):
        model.train()

        for batch_idx, batch in enumerate(tqdm(data['train'])):

            # Compute loss
            with torch.autocast(device_type=CONFIG.device, dtype=torch.float16, enabled=True):

                #if CONFIG.experiment in ['baseline']:
                if CONFIG.experiment in ['baseline']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    loss = F.cross_entropy(model(x), y)
                ######################################################

                elif CONFIG.experiment in ['activation_shaping_last', 'activation_shaping_one', 'activation_shaping_three']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    loss = F.cross_entropy(model(x), y)
                elif CONFIG.experiment in ['random']:
                    x, y = batch
                    x, y = x.to(CONFIG.device), y.to(CONFIG.device)
                    loss = F.cross_entropy(model(x), y)

                ######################################################

            # Optimization step
            scaler.scale(loss / CONFIG.grad_accum_steps).backward()

            if ((batch_idx + 1) % CONFIG.grad_accum_steps == 0) or (batch_idx + 1 == len(data['train'])):
                scaler.step(optimizer)
                optimizer.zero_grad(set_to_none=True)
                scaler.update()

        scheduler.step()

        # Test current epoch
        logging.info(f'[TEST @ Epoch={epoch}]')
        evaluate(model, data['test'])

        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'model': model.state_dict()
        }
        torch.save(checkpoint, os.path.join('record', CONFIG.experiment_name, 'last.pth'))

Main

In [11]:
def create_random_mask(layer_name = 'layer4.1.conv2', mask_out_ratio = 0.0):

    layer_output_shapes = {
        'layer1.0.conv1': (64, 56, 56),
        'layer1.0.relu': (64, 56, 56),
        'layer1.0.conv2': (64, 56, 56),
        'layer1.1.conv1': (64, 56, 56),
        'layer1.1.bn1': (64, 56, 56),
        'layer1.1.relu': (64, 56, 56),
        'layer1.1.conv2': (64, 56, 56),
        'layer1.1.bn2': (64, 56, 56),
        'layer2.0.conv1': (128, 28, 28),
        'layer2.0.relu': (128, 28, 28),
        'layer2.0.conv2': (128, 28, 28),
        'layer2.1.conv1': (128, 28, 28),
        'layer2.1.relu': (128, 28, 28),
        'layer2.1.conv2': (128, 28, 28),
        'layer3.0.conv1': (256, 14, 14),
        'layer3.0.relu': (256, 14, 14),
        'layer3.0.conv2': (256, 14, 14),
        'layer3.1.conv1': (256, 14, 14),
        'layer3.1.relu': (256, 14, 14),
        'layer3.1.conv2': (256, 14, 14),
        'layer4.0.conv1': (512, 7, 7),
        'layer4.0.relu': (512, 7, 7),
        'layer4.0.conv2': (512, 7, 7),
        'layer4.1.conv1': (512, 7, 7),
        'layer4.1.relu': (512, 7, 7),
        'layer4.1.conv2': (512, 7, 7)
    }

    # create a mask tensor with a given ratio of zeros
    print('mask_out_ratio: ',mask_out_ratio)
    layer_output_shape = layer_output_shapes[layer_name]

    rand_mat = torch.rand(layer_output_shape).to(CONFIG.device)
    mask = torch.where(rand_mat <= mask_out_ratio, 0.0, 1.0).to(CONFIG.device)
    return mask


# def main (mask_out_ratio=0.1):
def main ():
    # Load dataset
    data = load_data()

    # Load model
    if CONFIG.experiment in ['baseline']:
        model = BaseResNet18()
    ######################################################

#    elif CONFIG.experiment in ['activation_shaping_last']:
#        model = ASHResNet18()
#        model.define_network('ash_last', mask_out_ratio=0.1)
#    elif CONFIG.experiment in ['activation_shaping_one']:
#        model = ASHResNet18()
#        model.define_network('ash_one', mask_out_ratio=0.1)
#    elif CONFIG.experiment in ['activation_shaping_three']:
#        model = ASHResNet18()
#        model.define_network('ash_three', mask_out_ratio=0.1)

    elif CONFIG.experiment in ['random']:
        model = ASHResNet18()
        # module_placement = ['layer4.1.relu']
        # module_placement = ['layer3.1.conv2', 'layer4.0.conv2']
        module_placement = CONFIG.experiment_args['module_placement']
        mask_out_ratio = CONFIG.experiment_args['mask_out_ratio']

        for layer_name in module_placement:
            random_mask = create_random_mask(layer_name = layer_name, mask_out_ratio = mask_out_ratio)
            print('random_mask sum(): ', random_mask.sum())
            model.register_activation_shaping_hook(layer_name, random_mask)

    ######################################################

    model.to(CONFIG.device)

    if not CONFIG.test_only:
        train(model, data)
    else:
        evaluate(model, data['test'])

    if CONFIG.experiment in ['activation_shaping_last']:
        model.remove_activation_shaping_hook()
    elif CONFIG.experiment in ['activation_shaping_one', 'activation_shaping_three']:
        model.remove_activation_shaping_hooks()

### Baseline

Art painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': 'baseline/cartoon/',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='baseline', experiment_name='baseline/cartoon/', experiment_args='{}', dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 132MB/s]
100%|██████████| 19/19 [00:08<00:00,  2.31it/s]


Accuracy: 30.72 - Loss: 0.014738500830256492 



100%|██████████| 19/19 [00:07<00:00,  2.45it/s]


Accuracy: 41.89 - Loss: 0.013023637514879272 



...

Accuracy: 54.27 - Loss: 0.010764340816046597 



100%|██████████| 19/19 [00:07<00:00,  2.70it/s]


Accuracy: 54.22 - Loss: 0.010671518889252643 



100%|██████████| 19/19 [00:09<00:00,  2.01it/s]


Accuracy: 54.44 - Loss: 0.010674746455974342 



Art painting &rightarrow; Sketch

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': 'baseline/sketch/',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='baseline', experiment_name='baseline/sketch/', experiment_args='{}', dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


100%|██████████| 31/31 [00:12<00:00,  2.50it/s]


Accuracy: 10.66 - Loss: 0.016419021401826467 



100%|██████████| 31/31 [00:11<00:00,  2.64it/s]


Accuracy: 24.15 - Loss: 0.015490007752466822 



...

Accuracy: 40.06 - Loss: 0.01526401370771364 



100%|██████████| 31/31 [00:10<00:00,  2.98it/s]


Accuracy: 40.24 - Loss: 0.01506451536875458 



100%|██████████| 31/31 [00:11<00:00,  2.59it/s]


Accuracy: 40.47 - Loss: 0.015102852950359489 



Art painting &rightarrow; Photo

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'baseline',
    'experiment_name': 'baseline/photo/',
    'experiment_args': '{}',
    'dataset_args': { 'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

#args = parse_arguments()
args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu:
    CONFIG.device = 'cpu'
else:
    CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='baseline', experiment_name='baseline/photo/', experiment_args='{}', dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


100%|██████████| 14/14 [00:07<00:00,  1.94it/s]


Accuracy: 60.78 - Loss: 0.011621434103229088 



100%|██████████| 14/14 [00:05<00:00,  2.37it/s]


Accuracy: 79.70 - Loss: 0.006602727974246362 



...

Accuracy: 95.81 - Loss: 0.001072167762426916 



100%|██████████| 14/14 [00:05<00:00,  2.57it/s]


Accuracy: 95.81 - Loss: 0.0010833488375751558 



100%|██████████| 14/14 [00:05<00:00,  2.39it/s]


Accuracy: 95.87 - Loss: 0.001069648593418791 



## 1 - Activation Shaping module

### ASH module after last convolutional layer

Art painting &rightarrow; Cartoon

In [33]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/after_last_conv/cartoon/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/after_last_conv/cartoon/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
random_mask sum():  tensor(22611., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 19/19 [00:09<00:00,  1.96it/s]


Accuracy: 20.09 - Loss: 0.015888130451225176 



100%|██████████| 19/19 [00:06<00:00,  2.90it/s]


Accuracy: 27.73 - Loss: 0.014836208151062194 



...

Accuracy: 43.60 - Loss: 0.01158337647241537 



100%|██████████| 19/19 [00:05<00:00,  3.38it/s]


Accuracy: 43.17 - Loss: 0.01156649456292696 



100%|██████████| 19/19 [00:05<00:00,  3.35it/s]


Accuracy: 43.39 - Loss: 0.011544171936886302 



Art painting &rightarrow; Sketch

In [34]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/after_last_conv/sketch/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/after_last_conv/sketch/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
random_mask sum():  tensor(22611., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 31/31 [00:12<00:00,  2.48it/s]


Accuracy: 6.39 - Loss: 0.01705012803285229 



100%|██████████| 31/31 [00:10<00:00,  2.86it/s]


Accuracy: 10.38 - Loss: 0.016050937158153027 



...

Accuracy: 38.10 - Loss: 0.014110614155715705 



100%|██████████| 31/31 [00:09<00:00,  3.14it/s]


Accuracy: 38.28 - Loss: 0.014116268844876347 



100%|██████████| 31/31 [00:11<00:00,  2.66it/s]


Accuracy: 38.33 - Loss: 0.014158114187891594 



Art painting &rightarrow; Photo

In [35]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/after_last_conv/photo/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/after_last_conv/photo/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
random_mask sum():  tensor(22611., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 14/14 [00:05<00:00,  2.56it/s]


Accuracy: 33.29 - Loss: 0.014960164152933452 



100%|██████████| 14/14 [00:04<00:00,  2.86it/s]


Accuracy: 47.78 - Loss: 0.012349079040710084 



...

Accuracy: 91.32 - Loss: 0.003086106064255366 



100%|██████████| 14/14 [00:04<00:00,  3.17it/s]


Accuracy: 91.50 - Loss: 0.003095713953771991 



100%|██████████| 14/14 [00:04<00:00,  3.04it/s]


Accuracy: 91.50 - Loss: 0.0030852886374125223 



### ASH module after each convolutional layer

Art Painting &rightarrow; Cartoon

In [36]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/after_each_conv/cartoon/',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/after_each_conv/cartoon/', experiment_args={'module_placement': ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.1
random_mask sum():  tensor(180587., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
mask_out_ratio:  0.1
random_mask sum():  tensor(180685., device='cuda:0')
Insert activation shaping layer after  layer1.0.conv2 Conv

100%|██████████| 19/19 [00:08<00:00,  2.21it/s]


Accuracy: 15.87 - Loss: 0.015827962715471156 



100%|██████████| 19/19 [00:07<00:00,  2.50it/s]


Accuracy: 16.60 - Loss: 0.016055094758398298 



...

Accuracy: 21.08 - Loss: 0.015295659468442507 



100%|██████████| 19/19 [00:08<00:00,  2.33it/s]


Accuracy: 22.44 - Loss: 0.015255310938219975 



100%|██████████| 19/19 [00:07<00:00,  2.50it/s]


Accuracy: 21.08 - Loss: 0.01531952543266804 



### ASH module after every three convolutional layers


Art Painting &rightarrow; Cartoon

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/every_three_conv/cartoon/',
    'experiment_args': {'module_placement': ['layer1.0.conv1', 'layer1.1.conv2', 'layer2.1.conv1', 'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv2'], 'mask_out_ratio': 0.1},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

## 2 - Random Activation Maps

### Before running these cells: delete the corresponding last.pth file if present.

### mask out ratio = 0.4, module_placement: layer4.1.conv2


Art Painting &rightarrow; Cartoon

In [12]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case1/cartoon/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case1/cartoon/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 148MB/s]


mask_out_ratio:  0.4
random_mask sum():  tensor(15133., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 19/19 [00:06<00:00,  2.78it/s]


Accuracy: 20.39 - Loss: 0.01595527889785506 



...

Accuracy: 43.94 - Loss: 0.0115408024690257 



100%|██████████| 19/19 [00:06<00:00,  2.98it/s]


Accuracy: 43.94 - Loss: 0.011529821813513395 



100%|██████████| 19/19 [00:06<00:00,  2.92it/s]


Accuracy: 44.07 - Loss: 0.011507750956068267 



Art Painting &rightarrow; Sketch

In [13]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case1/sketch/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case1/sketch/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.4
random_mask sum():  tensor(15133., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 31/31 [00:12<00:00,  2.44it/s]


Accuracy: 4.53 - Loss: 0.017140563282538202 



100%|██████████| 31/31 [00:10<00:00,  2.95it/s]


Accuracy: 8.55 - Loss: 0.016083356926962448 



...

Accuracy: 39.53 - Loss: 0.013866907283834417 



100%|██████████| 31/31 [00:10<00:00,  3.04it/s]


Accuracy: 39.63 - Loss: 0.01388287950818185 



100%|██████████| 31/31 [00:11<00:00,  2.69it/s]


Accuracy: 39.83 - Loss: 0.013908702382364174 



Art Painting &rightarrow; Photo

In [14]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case1/photo/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case1/photo/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.4}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.4
random_mask sum():  tensor(15133., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 14/14 [00:06<00:00,  2.29it/s]


Accuracy: 33.35 - Loss: 0.015262928908456585 



100%|██████████| 14/14 [00:04<00:00,  3.16it/s]


Accuracy: 46.65 - Loss: 0.013173444470959509 



...

Accuracy: 90.90 - Loss: 0.003655705828509645 



100%|██████████| 14/14 [00:05<00:00,  2.38it/s]


Accuracy: 90.90 - Loss: 0.0036681195486805394 



100%|██████████| 14/14 [00:05<00:00,  2.80it/s]


Accuracy: 90.96 - Loss: 0.003649640458072731 



### mask out ratio = 0.55, module_placement: layer4.1.conv2

Art Painting &rightarrow; Cartoon

In [15]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Cartoon
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case2/cartoon/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case2/cartoon/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.55
random_mask sum():  tensor(11245., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 19/19 [00:06<00:00,  2.87it/s]


Accuracy: 19.88 - Loss: 0.015941636147352615 



100%|██████████| 19/19 [00:05<00:00,  3.27it/s]


Accuracy: 27.69 - Loss: 0.015007122232238587 



...

Accuracy: 43.90 - Loss: 0.01156081793251298 



100%|██████████| 19/19 [00:05<00:00,  3.40it/s]


Accuracy: 44.11 - Loss: 0.011530942935178711 



100%|██████████| 19/19 [00:05<00:00,  3.21it/s]


Accuracy: 44.03 - Loss: 0.011525774266532663 



Art Painting &rightarrow; Sketch

In [16]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Sketch
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case2/sketch/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case2/sketch/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.55
random_mask sum():  tensor(11245., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 31/31 [00:12<00:00,  2.50it/s]


Accuracy: 4.43 - Loss: 0.017113298943826155 



100%|██████████| 31/31 [00:10<00:00,  2.82it/s]


Accuracy: 8.22 - Loss: 0.016079276807740857 



...

Accuracy: 39.07 - Loss: 0.013798096793454609 



100%|██████████| 31/31 [00:11<00:00,  2.72it/s]


Accuracy: 39.07 - Loss: 0.013814138741127921 



100%|██████████| 31/31 [00:10<00:00,  2.98it/s]


Accuracy: 39.53 - Loss: 0.013823386153728186 



Art Painting &rightarrow; Photo

In [17]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case2/photo/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case2/photo/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.55}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.55
random_mask sum():  tensor(11245., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 14/14 [00:04<00:00,  2.91it/s]


Accuracy: 32.34 - Loss: 0.015373348761461451 



100%|██████████| 14/14 [00:04<00:00,  3.23it/s]


Accuracy: 45.75 - Loss: 0.013750154743651431 



...

Accuracy: 90.24 - Loss: 0.00407153774878222 



100%|██████████| 14/14 [00:05<00:00,  2.49it/s]


Accuracy: 90.06 - Loss: 0.004075139850199579 



100%|██████████| 14/14 [00:07<00:00,  1.99it/s]


Accuracy: 89.94 - Loss: 0.0040772847965092 



### mask out ratio = 0.75, module_placement: layer4.1.conv2

Art Painting &rightarrow; Cartoon

In [18]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case3/cartoon/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case3/cartoon/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'cartoon'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.75
random_mask sum():  tensor(6311., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 19/19 [00:08<00:00,  2.31it/s]


Accuracy: 18.69 - Loss: 0.01592104752316003 



100%|██████████| 19/19 [00:07<00:00,  2.54it/s]


Accuracy: 24.02 - Loss: 0.015258692738952898 



...

Accuracy: 43.13 - Loss: 0.01210388592078824 



100%|██████████| 19/19 [00:05<00:00,  3.22it/s]


Accuracy: 43.13 - Loss: 0.012061613818484362 



100%|██████████| 19/19 [00:05<00:00,  3.25it/s]


Accuracy: 43.00 - Loss: 0.012058371671637577 



Art Painting &rightarrow; Sketch

In [19]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case3/sketch/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case3/sketch/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'sketch'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.75
random_mask sum():  tensor(6311., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 31/31 [00:12<00:00,  2.50it/s]


Accuracy: 4.25 - Loss: 0.016792403131564115 



100%|██████████| 31/31 [00:12<00:00,  2.58it/s]


Accuracy: 6.13 - Loss: 0.01616382240854653 



...

Accuracy: 36.78 - Loss: 0.014144822215572754 



100%|██████████| 31/31 [00:11<00:00,  2.62it/s]


Accuracy: 37.47 - Loss: 0.01414790254235116 



100%|██████████| 31/31 [00:10<00:00,  3.01it/s]


Accuracy: 37.29 - Loss: 0.014151626581088615 



Art Painting &rightarrow; Photo

In [32]:
warnings.filterwarnings('ignore', category=UserWarning)

# Parse arguments
conf = { # Photo
    'seed': 0,
    'test_only': False,
    'cpu': False,
    'experiment': 'random',
    'experiment_name': 'random/case3/photo/',
    'experiment_args': {'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75},
    'dataset_args': {'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo' },
    'batch_size': 128,
    'epochs': 30,
    'num_workers': 5,
    'grad_accum_steps': 1
}

args = argparse.Namespace(**conf)
print(args)
CONFIG.update(vars(args))

# Setup output directory
CONFIG.save_dir = os.path.join('record', CONFIG.experiment_name)
os.makedirs(CONFIG.save_dir, exist_ok=True)

# Setup logging
logging.basicConfig(
    filename=os.path.join(CONFIG.save_dir, 'log.txt'),
    format='%(message)s',
    level=logging.INFO,
    filemode='a'
)

# Set experiment's device & deterministic behavior
if CONFIG.cpu: CONFIG.device = 'cpu'
else: CONFIG.device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(CONFIG.seed)
random.seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(mode=True, warn_only=True)

main()

Namespace(seed=0, test_only=False, cpu=False, experiment='random', experiment_name='random/case3/photo/', experiment_args={'module_placement': ['layer4.1.conv2'], 'mask_out_ratio': 0.75}, dataset_args={'text_root': 'data', 'images_root': 'data/kfold/PACS', 'source_domain': 'art_painting', 'target_domain': 'photo'}, batch_size=128, epochs=30, num_workers=5, grad_accum_steps=1)
mask_out_ratio:  0.75
random_mask sum():  tensor(6311., device='cuda:0')
Insert activation shaping layer after  layer4.1.conv2 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


100%|██████████| 14/14 [00:05<00:00,  2.63it/s]


Accuracy: 25.81 - Loss: 0.01563748799398274 



100%|██████████| 14/14 [00:04<00:00,  3.04it/s]


Accuracy: 38.32 - Loss: 0.014698316379935442 



...

Accuracy: 88.02 - Loss: 0.005243179719605132 



100%|██████████| 14/14 [00:06<00:00,  2.24it/s]


Accuracy: 87.90 - Loss: 0.0052470312503997436 



100%|██████████| 14/14 [00:06<00:00,  2.24it/s]


Accuracy: 87.96 - Loss: 0.005236520238979134 



## 3 - Adapting Activation Maps across Domains

## Ext. 2 - Binarization Ablation